### Basic Grammar Checker from GingerIt

In [1]:
from gingerit.gingerit import GingerIt

In [2]:
text = 'The smelt of fliwers bring back memories.'

parser = GingerIt()
text_ = parser.parse(text)
text_['result']
parser.parse('I is glad to has good frends like you')

{'text': 'I is glad to has good frends like you',
 'result': 'I am glad to have good friends like you',
 'corrections': [{'start': 22,
   'text': 'frends',
   'correct': 'friends',
   'definition': 'a person you know well and regard with affection and trust'},
  {'start': 13, 'text': 'has', 'correct': 'have', 'definition': None},
  {'start': 2, 'text': 'is', 'correct': 'am', 'definition': None}]}

### Formality Classifier (Work in Progress)

In [3]:
#Imports stuff and installs stuff takes a few secs to run
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import csv
import numpy as np
import spacy
import sklearn
import pickle
import re
from sklearn.preprocessing import MinMaxScaler
nltk.download('averaged_perceptron_tagger')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
string_punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
stop_words = set(stopwords.words('english'))
!python -m spacy download en_core_web_md

# Spacy model imported
nlp = spacy.load("en_core_web_md")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Steve\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
2021-12-02 18:16:19.931090: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-12-02 18:16:19.931115: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [4]:
#Active/passive voice

# function to check the type of sentence
# 0 refers to passive, 1 refers to active
def checkForSentType(inputSentence):   
    # running the model on sentence
    getDocFile = nlp(inputSentence)
    
    # getting the syntactic dependency 
    getAllTags = [token.dep_ for token in getDocFile]
    
    # checking for 'agent' tag
    checkPassiveTest = any(['agent' in sublist for sublist in getAllTags])
    
    # checking for 'nsubjpass' tag
    checkPassiveTestTwo = any(['nsubjpass' in sublist for sublist in getAllTags])
    if checkPassiveTest or checkPassiveTestTwo:
        return 0
    else:
        return 1


#Importing the list of formal/informal words
formal_words = pd.read_excel('formal_words.xlsx',header=None)
formal_list = formal_words[0].values.tolist()
informal_words = pd.read_excel('informal_words.xlsx',header=None)
informal_list = formal_words[0].values.tolist()


#Formal Pronouns
#Often third person
Formal_Pronouns = [
    "one","oneself","one's", 
    "who","whom","whomst","whose",
    "they","them","their","theirs","themself","themselves","theirself","theirselves",
    "it","its","itself",
    "he","him","himself","his",
    "she","her","herself","hers"
]

#Informal Pronouns
#Often first person
Informal_Pronouns = [
    "I","me","mine","mines","my","myself",
    "we","us","ourself","ourselves","our","ours",
    "you","your","yourself","yours","yourselves","y'all","yall","y'all's"
]

#Feature Extracter
def anvil_cleaner(data):
            
    #wored count
    data['Word Count']= data["text"].apply(lambda x: len(str(x).split()))
    
    #Counting number of formal pronouns
    data['Formal Pronoun'] = data["text"].apply(
        lambda x:len([w for w in str(x).lower().split() if w in Formal_Pronouns]))/data['Word Count']
        
    #Counting number of informal pronouns
    data['Informal Pronoun'] = data["text"].apply(
        lambda x:len([w for w in str(x).lower().split() if w in Informal_Pronouns]))/data['Word Count']
    
    #Counting number of contractions
    data['Contractions'] = data["text"].apply(lambda x: x.count("'"))/data['Word Count']
    
    #Identifying sentences with active voice
    data['Active Voice'] = data['text'].apply(checkForSentType)
    
    #Identifying sentences with passive voice (just opposite of active, since all sentences are active or passive)
    data['Passive Voice'] = 1 - data['text'].apply(checkForSentType)
        
    # Removing apostrophes so contractions are considered a single token
    data['clean_text'] = data['text'].apply(lambda x: re.sub("'", '', x))
    
    # Remove punctuation and stop words and lowercase the text
    data['clean_text'] = data['clean_text'].apply(lambda x: ' '.join(re.sub(
        r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words))

    #lemmatize
    #data['clean_text'] = data['clean_text'].apply(lambda x: ' '.join(lemmatizer.lemmatize(w) for w in x.split()))


    #now engineer the features the model expects
    
    #Formal words count (normalized to sentence length)
    #data['Formal Words'] = data["text"].apply(
    #    lambda x:len([w for w in str(x).split() if w in formal_list]))/data['Word Count']
        
    #Informal words count (normalized to sentence length)
    #data['Informal Words'] = data["text"].apply(
    #    lambda x:len([w for w in str(x).split() if w in informal_list]))/data['Word Count']
        
    #wored count
    data['Word Count']= data["text"].apply(lambda x: len(str(x).split()))


    data['Character count'] = data["text"].apply(lambda x: len(str(x))) 

    data["average characters per word"] =  data['Character count']/data['Word Count']

    data['stopword count'] = data["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))/data['Word Count']


    #feature for all the nowns in a text 
    from nltk import word_tokenize
    all_text_without_sw = ''
    for i in data.itertuples():
        all_text_without_sw = all_text_without_sw +  str(i.text)

    tokenized_all_text = word_tokenize(all_text_without_sw) #tokenize the text
    list_of_tagged_words = nltk.pos_tag(tokenized_all_text) #adding POS Tags to tokenized words

    set_pos  = (set(list_of_tagged_words)) # set of POS tags & words

    nouns = ['NN','NNS','NNP','NNPS'] #POS tags of nouns
    list_of_words = set(map(lambda tuple_2 : tuple_2[0], filter(lambda tuple_2 : tuple_2[1] in  nouns, set_pos)))
    #data['noun count'] = data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in list_of_words]) )/data['Word Count']

    # prnown count

    pronouns = ['PRP','PRP$','WP','WP$'] # POS tags of pronouns
    list_of_words = set(map(lambda tuple_2 : tuple_2[0], filter(lambda tuple_2 : tuple_2[1] in  pronouns, set_pos)))
    #data['pronoun count'] = data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in list_of_words]) )/data['Word Count']

    # count fo verbs

    verbs = ['VB','VBD','VBG','VBN','VBP','VBZ'] #POS tags of verbs
    list_of_words = set(map(lambda tuple_2 : tuple_2[0], filter(lambda tuple_2 : tuple_2[1] in  verbs, set_pos)))
    #data['verb count'] = data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in list_of_words]) )/data['Word Count']


    #adverb count

    adverbs = ['RB','RBR','RBS','WRB'] #POS tags of adverbs
    list_of_words = set(map(lambda tuple_2 : tuple_2[0], filter(lambda tuple_2 : tuple_2[1] in  adverbs, set_pos)))
    data['adverb count'] = data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in list_of_words]) )/data['Word Count']

    #Adjective count

    adjectives = ['JJ','JJR','JJS'] #POS tags of adjectives
    list_of_words = set(map(lambda tuple_2 : tuple_2[0], filter(lambda tuple_2 : tuple_2[1] in  adjectives, set_pos)))
    #data['adjective count'] = data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in list_of_words]) )/data['Word Count']


    data['punctuation count'] = data['clean_text'].apply(lambda x: len([w for w in str(x) if w in string_punctuation]))/data['Word Count']


    data['mean sentance length'] = data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
    return data



In [5]:
#Input anvil sentence text in sentences list
sentences = ["How's it going?",
            "It is our honest opinion that our organization is not properly positioned to invest in that platform.",
            "We have reviewed your application and, unfortunately, we have decided to move forward with another applicant for this position.",
            "Hello I am very nice to meet you!",
            "I need help with my project."]

#Converts that to a dataframe
data = pd.DataFrame(sentences,columns=['text'])

In [6]:
#Load in trained pickle model (Logistic regression)
with open('formality_model.pkl', 'rb') as f:
    lr_loaded = pickle.load(f)
clean_data = anvil_cleaner(data)

#Transforms data to be inserted into model (drops text and scales)
clean_data = clean_data.drop(['text', 'clean_text'], axis=1)
cols = clean_data.columns
scaler = MinMaxScaler()
clean_data = scaler.fit_transform(clean_data)
clean_data = pd.DataFrame(clean_data, columns=[cols])

#Get probabilities of formalities (0 is informal, 1 is formal)
lr_loaded.predict_proba(clean_data)[:,1]

array([9.99225633e-01, 1.00000000e+00, 1.00000000e+00, 7.20758102e-01,
       1.41217275e-04])

## RNN for Informal to Formal

In [7]:
from torchtext.legacy.data import Field, TabularDataset, BucketIterator

In [8]:
import torch
import spacy
from torchtext.data.metrics import bleu_score
import sys


def translate_sentence(model, sentence, informal, formal, device, max_length=50):
    # print(sentence)

    # sys.exit()

    # Load german tokenizer
    spacy_en = spacy.load("en_core_web_sm")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_en(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, informal.init_token)
    tokens.append(informal.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [informal.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)

    outputs = [formal.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == formal.vocab.stoi["<eos>"]:
            break

    translated_sentence = [formal.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence[1:]


def bleu(data, model, informal, formal, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, informal, formal, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import spacy
import random
from torch.utils.tensorboard import SummaryWriter

In [10]:
spacy_en = spacy.load('en_core_web_sm')

def tokenizer_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

informal = Field(tokenize=tokenizer_en, 
                 lower=True, 
                 init_token='<sos>', 
                 eos_token='<eos>')
formal = Field(tokenize=tokenizer_en, 
               lower=True, 
               init_token='<sos>', 
               eos_token='<eos>')

new_fields = {'informal': ("src", informal), 'formal': ("trg", formal)}

train_data, validation_data, test_data = TabularDataset.splits(path='data', 
                                              train='train.csv', validation='valid.csv', 
                                              test='test.csv', 
                                              format='csv', 
                                              fields=new_fields)

informal.build_vocab(train_data, max_size=10000, min_freq=2)
formal.build_vocab(train_data, max_size=10000, min_freq=2)

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, drop_p):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.dropout = nn.Dropout(drop_p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=drop_p)
        
    def forward(self, x):
        # x shape: (seq_length, N)
#         print("x encode", x.shape)
        
        embedding = self.dropout(self.embedding(x))
        outputs, (hidden, cell) = self.rnn(embedding)
        
#         print("embedding encode", embedding.shape)
#         print("hidden encode", hidden.shape)
#         print("cell encode", cell.shape)
        
        return hidden, cell

In [12]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, drop_p):
        super(Decoder, self).__init__()
        
        #output_size=input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.dropout = nn.Dropout(drop_p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, 
                           hidden_size, 
                           num_layers, 
                           dropout=drop_p)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden, cell):
        # x shape : N, want (1, N)
        # translating one word at a time
#         print("x before decode", x.shape)
        x = x.unsqueeze(0)
#         print("x after decode", x.shape)
        
        embedding = self.dropout(self.embedding(x))
#         print('hidden:', hidden.shape)
#         print('cell:', cell.shape)
        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        
        predictions = self.fc(outputs)
        
        predictions = predictions.squeeze(0)
        
        return predictions, hidden, cell

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, source, target, teacher_force_ratio=0.5):
        # teacher determines chance to use predicted translation rather than target translation
        # prevents overtraining on data, test time it will see very diff words from train
        
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(formal.vocab)
        
        outputs = torch.zeros(target_len, 
                              batch_size, 
                              target_vocab_size).to(device)
        
        # run things into encoder to get hidden and cell, then run those through decoder
        hidden, cell = self.encoder(source)
        
        # start token
        x = target[0]
        
        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            
            outputs[t] = output
            
            best_guess = output.argmax(1)
            
            x = target[t] if random.random() < teacher_force_ratio else best_guess
            
        return outputs

In [14]:
# training hyperparam

num_epochs = 20
learning_rate = 0.001
batch_size = 64

# model hyperparam

load_model = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(informal.vocab)
input_size_decoder = len(formal.vocab)
output_size = len(formal.vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5

# tensorboard
writer = SummaryWriter(f'runs/loss_plot')
step = 0

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, 
                                                                      validation_data, 
                                                                      test_data), 
                                                                      batch_size=batch_size, 
                                                                      sort_within_batch=True, 
                                                                      sort_key= lambda x: len(x.src), 
                                                                      device=device)

encoder_net = Encoder(input_size_encoder, 
                      encoder_embedding_size, 
                      hidden_size, 
                      num_layers, 
                      enc_dropout).to(device)

decoder_net = Decoder(input_size_decoder, 
                      decoder_embedding_size, 
                      hidden_size, 
                      output_size, 
                      num_layers, 
                      dec_dropout).to(device)

model = Seq2Seq(encoder_net, decoder_net).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = formal.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

if load_model:
    load_checkpoint(torch.load('my_checkpoint.pth.tar'), model, optimizer)
    
# for epoch in range(num_epochs):
#     print(f'Epoch [{epoch}/{num_epochs}]')
    
#     checkpoint = {'state_dict':model.state_dict(), 'optimizer':optimizer.state_dict()}
#     save_checkpoint(checkpoint)
    
#     for batch_idx, batch in enumerate(train_iterator):
#         inp_data = batch.src.to(device)
#         target = batch.trg.to(device)
        
#         output = model(inp_data, target)
#         # output: (trg_len, batch_size, output_dim)
        
#         output = output[1:].reshape(-1, output.shape[2])
#         target = target[1:].reshape(-1)
        
#         optimizer.zero_grad()
#         loss = criterion(output, target)
        
#         loss.backward()
        
#         torch.nn.utils.clip_grad_norm(model.parameters(), max_norm=1)
#         optimizer.step()
        
#         writer.add_scalar('Training Loss', loss, global_step=step)
#         step += 1

=> Loading checkpoint


In [15]:
def to_formal(sentence):
    lst = translate_sentence(model, sentence, informal, formal, device)
    final = ""
    for i in range(len(lst)-1):
        if i == len(lst)-2:
            final = final.strip()
        final = final + lst[i] + " "
    final = final.strip()
    return final.capitalize()

In [16]:
to_formal("How's life?")

'How is anyway?'

## Sentence Level Probability

In [18]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.tokenize import sent_tokenize

model_translate = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer_translate = GPT2Tokenizer.from_pretrained('gpt2')

In [19]:
def score(tokens_tensor):
    loss=model_translate(tokens_tensor, labels=tokens_tensor)[0]
    return np.exp(loss.cpu().detach().numpy())

def GPT2_Sentence_Scorer(text):
    scores = []
    sentences = sent_tokenize(text)
    for sent in sentences:
        scores.append((score(tokenizer_translate.encode(sent, add_special_tokens=False, return_tensors="pt")), sent))
    return scores

## Translation Models From HuggingFace

In [28]:
import numpy as np
import pandas as pd

import json
import requests
import time
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification
from transformers import DistilBertTokenizerFast

In [21]:
API_TOKEN="api_oMMRDkimUuBHSmsDNophzfNCmbzfimIFRl"

models_dictionary = {'English to Spanish':"https://api-inference.huggingface.co/models/Helsinki-NLP/opus-mt-en-es",
                    'Spanish to English':"https://api-inference.huggingface.co/models/Helsinki-NLP/opus-mt-es-en",
                    'English to French':"https://api-inference.huggingface.co/models/Helsinki-NLP/opus-mt-en-fr",
                    'French to English':"https://api-inference.huggingface.co/models/Helsinki-NLP/opus-mt-fr-en",
                    'English to Chinese':"https://api-inference.huggingface.co/models/Helsinki-NLP/opus-mt-en-zh",
                    'Chinese to English':"https://api-inference.huggingface.co/models/Helsinki-NLP/opus-mt-zh-en"}

def query(payload, API_URL, headers):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))

In [ ]:
import anvil.server

anvil.server.connect("WY53G5FKUA3OMQSIHJ2GCQLL-NVAGOPE44XRT7GXJ")

@anvil.server.callable
def grammar_checker(text):
    if parser.parse(text)['corrections']:
        return parser.parse(text)['result']
    else:
        return 'No corrections needed.'
    
@anvil.server.callable    
def formality_checker(text):
    sentences = sent_tokenize(text)
    data = pd.DataFrame(sentences,columns=['text'])
    clean_data = anvil_cleaner(data)

    #Transforms data to be inserted into model (drops text and scales)
    clean_data = clean_data.drop(['text', 'clean_text'], axis=1)
    cols = clean_data.columns
    scaler = MinMaxScaler()
    clean_data = scaler.fit_transform(clean_data)
    clean_data = pd.DataFrame(clean_data, columns=[cols])

    #Get probabilities of formalities (0 is informal, 1 is formal)
    vals = list((zip(lr_loaded.predict_proba(clean_data)[:,1], sentences)))
    output = ''
    for val in vals:
        if val[0] > 0.5:
            output = output + "Formal:  " + val[1] + '\n'
        else:
            output = output + "Informal:  " + val[1] + '\n'
        output = output + '\n'
    return output

@anvil.server.callable  
def rnn_informal_formal(x):
    return to_formal(x)
    
@anvil.server.callable
def translate(text, language):
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    API_URL = models_dictionary[language]
    result = query({"inputs":f'{text}',}, API_URL, headers)
    while 'error' in result:
        time.sleep(20)
        result = query({"inputs":f'{text}',}, API_URL, headers)
        print(result)
    return result[0]['translation_text']

@anvil.server.callable
def nat_sounding(text):
    list_of_scores = GPT2_Sentence_Scorer(text)
    good, okay, bad = [], [], []
    for sent in list_of_scores:
        if sent[0] > 100 and sent[0] < 200:
            okay.append(sent[1] + '\n')
        elif sent[0] >= 200:
            bad.append(sent[1] + '\n')
        else:
            good.append(sent[1] + '\n')
    if len(okay) == 0:
        okay.append('None')
    if len(bad) == 0:
        okay.append('None')
    print(list_of_scores)
    return (good,okay,bad)
    
anvil.server.wait_forever()

Disconnecting from previous connection first...
Connecting to wss://anvil.works/uplink
Anvil websocket closed (code 1000, reason=b'')
Anvil websocket open
Connected to "Default environment" as SERVER
{'error': 'Model Helsinki-NLP/opus-mt-en-es is currently loading', 'estimated_time': 20}
[{'translation_text': 'Hola.'}]
{'error': 'Model Helsinki-NLP/opus-mt-en-fr is currently loading', 'estimated_time': 20}
[{'translation_text': 'Bonjour.'}]
{'error': 'Model Helsinki-NLP/opus-mt-en-zh is currently loading', 'estimated_time': 20}
[{'translation_text': '喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂 喂'}]
[(166.31491, 'Subsequently the team parsed, cleaned and normalized data and to optimize it for the advanced deep learning powered language models they use to generate their suggestions.'), (39.54744, 'The final product is displayed on an interactive, easy to use platform where users can write their messages from scratch, receiving feedback as they go.'), (7806.339, 'This not most sounding na

In [ ]:
!pip install anvil-uplink